In [1]:
import tensorflow as tf
print(tf.__version__)

2.0.0


# 单词和字符串的one-hot编码

In [2]:
# 6-1单词级的one-hot编码
import numpy as np

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# 构建数据中所有标记的索引
token_index = {}
for sample in samples:
    
    # 利用split方法对样本进行分词
    for word in sample.split():
        if word not in token_index:
            # 为每个唯一单词指定一个索引
            token_index[word] = len(token_index) + 1
            # 没有为索引编好0指定单词


# 只考虑每个样本前max_length个单词
max_length = 10

# 结果保存在results中
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1.

In [3]:
# 6-2字符级的one-hot编码
import string

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
characters = string.printable  
token_index = dict(zip(characters, range(1, len(characters) + 1)))

max_length = 50
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, character in enumerate(sample[:max_length]):
        index = token_index.get(character)
        results[i, j, index] = 1.

In [4]:
# 6-3使用keras实现单词级的one-hot编码
from tensorflow.keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# 创建一个分词器tokenizer，设置为只考虑前10000个常见的单词
tokenizer = Tokenizer(num_words=1000)
# 构建单词索引
tokenizer.fit_on_texts(samples)

# 将字符串转换为整数索引组成的列表
sequences = tokenizer.texts_to_sequences(samples)

one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')

# 找回单词索引
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 9 unique tokens.


In [5]:
# 6-4使用散列技巧的单词级的one-hot编码
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

dimensionality = 1000
max_length = 10

results = np.zeros((len(samples), max_length, dimensionality))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:

        index = abs(hash(word)) % dimensionality
        results[i, j, index] = 1.

# 词嵌入

In [5]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(1000, 64)

In [8]:
# 6-6加载IMDB数据，用于Embedding层
from tensorflow.keras.datasets import imdb
from tensorflow.keras import preprocessing

# 作为特征的单词个数
max_features = 10000

maxlen = 20

# 加载数据
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# 将整数列表转换为形状(samples, maxlen)的二维整数张量
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

model = Sequential()

model.add(Embedding(10000, 8, input_length=maxlen))

model.add(Flatten())


model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten (Flatten)            (None, 160)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________
Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 3s 152us/sample - loss: 0.6722 - acc: 0.6136 - val_loss: 0.6249 - val_acc: 0.6946
Epoch 2/10
20000/20000 [==============================] - 2s 96us/sample - loss: 0.5466 - acc: 0.7506 - val_loss: 0.5285 - val_acc: 0.7272
Epoch 3/10
20000/20000 [==============================